In [8]:
import pandas as pd
import datetime
import os
def generate_sales_report(df, enriched_list, output_file='output/sales_report.txt'):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    df['Date'] = pd.to_datetime(df['Date'])

    total_revenue = df['TotalSales'].sum()
    avg_order = df['TotalSales'].mean()

    line_eq = "=" * 44
    line_dash = "-" * 44

    report = []

    # HEADER
    report.append(line_eq)
    report.append(f"{'SALES ANALYTICS REPORT':^44}")
    report.append(f"Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'):^44}")
    report.append(f"Records Processed: {len(df):^44}")
    report.append(line_eq + "\n")

    # OVERALL SUMMARY
    report.append("OVERALL SUMMARY")
    report.append(line_dash)
    report.append(f"Total Revenue:        ₹{total_revenue:,.2f}")
    report.append(f"Total Transactions:   {len(df)}")
    report.append(f"Average Order Value:  ₹{avg_order:,.2f}")
    report.append(
        f"Date Range:           {df['Date'].min().date()} to {df['Date'].max().date()}\n"
    )

    # REGION-WISE PERFORMANCE
    report.append("REGION-WISE PERFORMANCE")
    report.append(line_dash)
    report.append(f"{'Region':<10} {'Sales':<15} {'% of Total':<12} {'Transactions'}")

    region_df = (
        df.groupby('Region')
        .agg(TotalSales=('TotalSales', 'sum'),
             Transactions=('TransactionID', 'count'))
        .sort_values('TotalSales', ascending=False)
    )

    for reg, row in region_df.iterrows():
        percent = (row['TotalSales'] / total_revenue) * 100
        report.append(
            f"{reg:<10} ₹{row['TotalSales']:<14,.0f} {percent:<11.2f}% {row['Transactions']}"
        )

    report.append("")

    # TOP PRODUCTS
    report.append("TOP 5 PRODUCTS")
    report.append(line_dash)

    top_products = (
        df.groupby('ProductName')
        .agg(Quantity=('Quantity', 'sum'),
             Revenue=('TotalSales', 'sum'))
        .sort_values('Revenue', ascending=False)
        .head(5)
    )

    for i, (name, row) in enumerate(top_products.iterrows(), 1):
        report.append(
            f"{i}. {name[:25]:<25} | Qty: {int(row['Quantity']):<5} | Rev: ₹{row['Revenue']:,.0f}"
        )

    report.append("")

    # API ENRICHMENT SUMMARY
    success_count = sum(1 for tx in enriched_list if tx.get('API_Category'))
    failed = list(
        {tx.get('ProductName') for tx in enriched_list
         if not tx.get('API_Category') and tx.get('ProductName')}
    )

    report.append("API ENRICHMENT SUMMARY")
    report.append(line_dash)
    report.append(f"Total products processed: {len(enriched_list)}")
    report.append(
        f"Success rate:            {(success_count / len(enriched_list)) * 100:.2f}%"
    )

    if failed:
        report.append(f"Failed Enrichment:       {', '.join(failed[:3])}...")

    # WRITE FILE
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(report))

    print(f"REPORT GENERATED SUCCESSFULLY AT: {output_file}")

In [9]:
transactions_data = pd.read_csv('processed_sales.csv')
api_data = pd.read_csv('enriched_data.csv').to_dict('records')

# Create TotalSales if missing
if 'TotalSales' not in transactions_data.columns:
    transactions_data['TotalSales'] = (
        transactions_data['Quantity'] * transactions_data['UnitPrice']
    )


In [10]:
transactions_data.head()


,TransactionID,Date,ProductID,ProductName,Quantity,UnitPrice,CustomerID,Region,TotalSales
0,T018,2024-12-29,P107,USB Cable,8,173.0,C009,South,1384.0
1,T023,2024-12-09,P109,Wireless Mouse,9,523.0,C022,North,4707.0
2,T059,2024-12-29,P102,MouseWireless,4,1056.0,C010,South,4224.0
3,T035,2024-12-08,P102,Mouse,4,431.0,C011,North,1724.0
4,T061,2024-12-10,P109,Wireless Mouse,2,775.0,C009,North,1550.0


In [11]:
generate_sales_report(
    df=transactions_data,
    enriched_list=api_data,
    output_file='output/sales_report.txt'
)


REPORT GENERATED SUCCESSFULLY AT: output/sales_report.txt
